---
<center><h1>Basic intro into pandas</h1></center> 
---
---

<center><h2>Working with pandas DataFrames: grouping</h2></center>
---

## Table of Contents

- [Work with pandas DataFrames: grouping](#Work-with-pandas-DataFrames:-grouping)
    * [Splitting of a DataFrame into groups](#Splitting-of-a-DataFrame-into-groups)
    * [Selection and filtering](#Selection-and-filtering)
    * [Aggregation and function application](#Aggregation-and-function-application)
    - [*Exercise 1*](#Exercise-1)

In [122]:
import pandas as pd
import numpy as np
import random

## Work with pandas DataFrames: grouping

[[back to top]](#Table-of-Contents)

We will a werather station dataset so that we can see what is going on when we group.

In [183]:
from arcgis.features import GeoAccessor, GeoSeriesAccessor
#Import a COVID data layer.  This layer contains the updated stats for each county in the United States
from arcgis.features import FeatureLayer
mylayer = FeatureLayer(("https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NOAA_METAR_current_wind_speed_direction_v1/FeatureServer/0"))
sdf2 = pd.DataFrame.spatial.from_layer(mylayer)
sdf2.head(5)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE
0,1,YPDN,2021-02-24 16:06:00,Darwin (Civ/Mil),Australia,32.0,77.0,75.2,94.0,120.0,9,0,NaN,6000.0,1006.9,"Broken Clouds at 2100 meters AGL, Broken Cloud...",Rain,None,NaN,-12.42,130.90,"{""x"": 14571721.344800003, ""y"": -1393544.816599..."
1,2,PAJZ,2021-02-24 16:05:00,Koliganek,"Alaska, United States Of America",80.0,26.6,26.6,100.0,120.0,24,0,14.7,4000.0,1003.0,"Few Clouds at 490 meters AGL, Overcast Cloud D...","Light Snow, Mist",* Maintenance REQUIRED - readings may be compr...,NaN,59.73,-157.27,"{""x"": -17507216.3171, ""y"": 8339869.139899999, ..."
2,3,PAIW,2021-02-24 16:05:00,Wales,"Alaska, United States Of America",6.0,-7.6,-11.2,84.0,70.0,56,0,-37.8,1600.0,1007.8,Sky Clear below 3700 meters AGL,"Light Snow, Mist",Hourly Precipitation Amount: trace,NaN,65.62,-168.10,"{""x"": -18712806.4023, ""y"": 9773610.204599999, ..."
3,4,MMCV,2021-02-24 16:05:00,Ciudad Victoria,Mexico,230.0,71.6,59.0,65.0,140.0,24,50,NaN,13000.0,1016.6,Clear Skies,No significant weather present at this time.,None,76.2,23.72,-98.97,"{""x"": -11017290.0038, ""y"": 2719325.819600001, ..."
4,5,KSMN,2021-02-24 16:05:00,Salmon/Lemhi,"Idaho, United States Of America",1228.0,26.6,21.2,80.0,0.0,0,0,52.3,2800.0,1024.7,"Broken Clouds at 910 meters AGL, Overcast Clou...",Light Snow,Hourly Precipitation Amount: trace,NaN,45.12,-113.87,"{""x"": -12675950.4166, ""y"": 5640432.8549000025,..."


In [184]:
sdfUSA = sdf2.loc[sdf2["COUNTRY"].apply(lambda x: "United States Of America" in str(x))]
sdfUSA["State"] = sdfUSA['COUNTRY'].apply(lambda x: str(x).split(",")[0])
sdfUSA["COUNTRY"] = sdfUSA['COUNTRY'].apply(lambda x: str(x).split(",")[1])
sdfUSA

D66140B0-0277-45FF-AE18-A787BE3D4AAF:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D66140B0-0277-45FF-AE18-A787BE3D4AAF:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE,State
1,2,PAJZ,2021-02-24 16:05:00,Koliganek,United States Of America,80.0,26.6,26.6,100.0,120.0,24,0,14.7,4000.0,1003.0,"Few Clouds at 490 meters AGL, Overcast Cloud D...","Light Snow, Mist",* Maintenance REQUIRED - readings may be compr...,NaN,59.73,-157.27,"{""x"": -17507216.3171, ""y"": 8339869.139899999, ...",Alaska
2,3,PAIW,2021-02-24 16:05:00,Wales,United States Of America,6.0,-7.6,-11.2,84.0,70.0,56,0,-37.8,1600.0,1007.8,Sky Clear below 3700 meters AGL,"Light Snow, Mist",Hourly Precipitation Amount: trace,NaN,65.62,-168.10,"{""x"": -18712806.4023, ""y"": 9773610.204599999, ...",Alaska
4,5,KSMN,2021-02-24 16:05:00,Salmon/Lemhi,United States Of America,1228.0,26.6,21.2,80.0,0.0,0,0,52.3,2800.0,1024.7,"Broken Clouds at 910 meters AGL, Overcast Clou...",Light Snow,Hourly Precipitation Amount: trace,NaN,45.12,-113.87,"{""x"": -12675950.4166, ""y"": 5640432.8549000025,...",Idaho
5,6,KKLS,2021-02-24 16:05:00,Kelso Longveiw,United States Of America,4.0,33.8,32.0,93.0,140.0,7,0,29.5,1200.0,1035.9,Overcast Cloud Deck at 60 meters AGL,Mist,None,NaN,46.12,-122.90,"{""x"": -13681165.418499999, ""y"": 5799600.201899...",Washington
6,7,KCIU,2021-02-24 16:05:00,Chippewa Intl,United States Of America,242.0,30.2,28.4,93.0,30.0,19,0,20.7,1200.0,1005.7,"Broken Clouds at 240 meters AGL, Overcast Clou...","Light Snow, Mist",Hourly Precipitation Amount: trace,NaN,46.25,-84.47,"{""x"": -9403157.3873, ""y"": 5820502.805600002, ""...",Michigan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5044,5045,KMBT,2021-02-24 14:55:00,Murfreesboro,United States Of America,187.0,59.0,32.0,36.0,210.0,17,0,NaN,16000.0,1017.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,NaN,35.88,-86.38,"{""x"": -9615777.6147, ""y"": 4284122.0951000005, ...",Tennessee
5045,5046,KVEL,2021-02-24 14:53:00,Vernal,United States Of America,1604.0,26.1,8.1,46.0,360.0,11,0,18.4,16000.0,1018.9,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,3-Hour Pressure Increasing: 0.6 mb,NaN,40.45,-109.52,"{""x"": -12191710.6317, ""y"": 4931552.160499997, ...",Utah
5046,5047,KNIP,2021-02-24 14:53:00,Jacksonville Nas,United States Of America,3.0,62.1,51.1,67.0,0.0,0,0,NaN,16000.0,1023.4,Scattered Clouds at 1000 meters AGL,No significant weather present at this time.,3-Hour Pressure Increasing: 1.5 mb,NaN,30.23,-81.67,"{""x"": -9091462.8131, ""y"": 3533148.591300003, ""...",Florida
5047,5048,KJER,2021-02-24 14:53:00,Jerome County,United States Of America,1226.0,24.1,19.0,81.0,290.0,9,0,17.0,16000.0,1022.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,3-Hour Pressure Increasing: 1.2 mb,NaN,42.72,-114.45,"{""x"": -12740515.7213, ""y"": 5269449.638999999, ...",Idaho


In [185]:
sdfUSA.dtypes

OBJECTID                 int64
ICAO                    object
OBS_DATETIME    datetime64[ns]
STATION_NAME            object
COUNTRY                 object
ELEVATION              float64
TEMP                   float64
DEW_POINT              float64
R_HUMIDITY             float64
WIND_DIRECT            float64
WIND_SPEED               int64
WIND_GUST                int64
WIND_CHILL             float64
VISIBILITY             float64
PRESSURE               float64
SKY_CONDTN              object
WEATHER                 object
REMARKS                 object
HEAT_INDEX             float64
LATITUDE               float64
LONGITUDE              float64
SHAPE                 geometry
State                   object
dtype: object

So, let’s shortly describe what you will know after reading the blog post:
1.	how to split a DataFrame into a group;
2.	how select a group in the grouped DataFrame and filter the grouping data;
3.	how aggregate grouping data and how apply many function at once to the grouping data;


### Splitting of a DataFrame into groups

[[back to top]](#Table-of-Contents)

pandas DataFrames can be split on any of their axes. Grouping denotes the providence of a mapping of labels to group names. Method `groupby()` is provided in pandas for grouping. The `groupby()` function returns a `GroupBy` object, but essentially describes how the rows of the original data set has been split. 


In [231]:
sdfUSA_grouped = sdfUSA.groupby('State')
sdfUSA_grouped

In [232]:
sdfUSA_grouped['COUNTRY']

Use `list()` to show what a grouping looks like

In [233]:
list(sdfUSA_grouped)

[('Alabama',       OBJECTID  ...    State
1132      1133  ...  Alabama
1147      1148  ...  Alabama
1150      1151  ...  Alabama
1154      1155  ...  Alabama
1161      1162  ...  Alabama
1180      1181  ...  Alabama
1345      1346  ...  Alabama
1372      1373  ...  Alabama
1381      1382  ...  Alabama
1450      1451  ...  Alabama
1470      1471  ...  Alabama
1532      1533  ...  Alabama
1535      1536  ...  Alabama
1607      1608  ...  Alabama
1721      1722  ...  Alabama
1728      1729  ...  Alabama
1818      1819  ...  Alabama
1929      1930  ...  Alabama
2015      2016  ...  Alabama
2019      2020  ...  Alabama
2222      2223  ...  Alabama
2297      2298  ...  Alabama
2314      2315  ...  Alabama
2427      2428  ...  Alabama
2442      2443  ...  Alabama
2471      2472  ...  Alabama
2514      2515  ...  Alabama
2515      2516  ...  Alabama
2563      2564  ...  Alabama
2814      2815  ...  Alabama
2822      2823  ...  Alabama
2944      2945  ...  Alabama
2966      2967  ...  Alabama
3

As it can be easily seen, the `GroupBy` object is a dictionary whose keys are the computed unique groups and corresponding values being the axis labels belonging to each group.

In [234]:
for key, value in list(sdfUSA_grouped)[:3]:
# if you want to see more values uncomment row below and comment the previous row
#for key, value in short_grouped:
    print (key)
    print (value)

Alabama
      OBJECTID  ...    State
1132      1133  ...  Alabama
1147      1148  ...  Alabama
1150      1151  ...  Alabama
1154      1155  ...  Alabama
1161      1162  ...  Alabama
1180      1181  ...  Alabama
1345      1346  ...  Alabama
1372      1373  ...  Alabama
1381      1382  ...  Alabama
1450      1451  ...  Alabama
1470      1471  ...  Alabama
1532      1533  ...  Alabama
1535      1536  ...  Alabama
1607      1608  ...  Alabama
1721      1722  ...  Alabama
1728      1729  ...  Alabama
1818      1819  ...  Alabama
1929      1930  ...  Alabama
2015      2016  ...  Alabama
2019      2020  ...  Alabama
2222      2223  ...  Alabama
2297      2298  ...  Alabama
2314      2315  ...  Alabama
2427      2428  ...  Alabama
2442      2443  ...  Alabama
2471      2472  ...  Alabama
2514      2515  ...  Alabama
2515      2516  ...  Alabama
2563      2564  ...  Alabama
2814      2815  ...  Alabama
2822      2823  ...  Alabama
2944      2945  ...  Alabama
2966      2967  ...  Alabama
3037  

Besides, you may to see the first and the last item of each group

In [235]:
sdfUSA_grouped.first().head(10)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE
State,,,,,,,,,,,,,,,,,,,,,,
Alabama,1133,KSXS,2021-02-24 15:58:00,Ft Rucker/Shell,United States Of America,120.0,64.6,43.5,46.0,240.0,9,0,NaN,14000.0,1022.0,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,78.8,31.37,-85.85,"{""x"": -9556778.2846, ""y"": 3680894.4886000007, ..."
Alaska,2,PAJZ,2021-02-24 16:05:00,Koliganek,United States Of America,80.0,26.6,26.6,100.0,120.0,24,0,14.7,4000.0,1003.0,"Few Clouds at 490 meters AGL, Overcast Cloud D...","Light Snow, Mist",* Maintenance REQUIRED - readings may be compr...,NaN,59.73,-157.27,"{""x"": -17507216.3171, ""y"": 8339869.139899999, ..."
Arizona,1110,KNOZ,2021-02-24 15:59:00,Wellton,United States Of America,78.0,60.1,27.0,28.0,150.0,15,0,30.3,16000.0,1016.9,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,32.53,-114.50,"{""x"": -12746081.695799999, ""y"": 3833084.206799..."
Arkansas,1140,KROG,2021-02-24 15:58:00,Rogers,United States Of America,407.0,53.1,35.1,50.0,10.0,15,0,NaN,16000.0,1015.6,Overcast Cloud Deck at 1000 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,36.37,-94.10,"{""x"": -10475164.0836, ""y"": 4351652.965599999, ..."
California,14,KNSI,2021-02-24 16:04:00,San Nicolas Isla,United States Of America,161.0,54.0,53.1,97.0,350.0,11,0,66.8,1200.0,1016.9,"Scattered Clouds at 90 meters AGL, Broken Clou...",Mist,* Maintenance REQUIRED - readings may be compr...,NaN,33.23,-119.45,"{""x"": -13297113.1753, ""y"": 3925872.5605000034,..."
Colorado,1112,KMYP,2021-02-24 15:59:00,Monarch Pass,United States Of America,3629.0,10.4,6.8,85.0,240.0,31,0,-8.2,16000.0,1020.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,38.48,-106.32,"{""x"": -11835488.2611, ""y"": 4647457.398199998, ..."
Connecticut,1445,KGON,2021-02-24 15:56:00,Groton/New Londo,United States Of America,2.0,46.0,35.1,65.0,210.0,19,0,40.6,16000.0,1016.9,Sky Clear below 3700 meters AGL,No significant weather present at this time.,Wind Shift At 15:30Z,NaN,41.33,-72.05,"{""x"": -8020569.3116999995, ""y"": 5061139.118699..."
Delaware,1485,KDOV,2021-02-24 15:56:00,Dover Afb,United States Of America,6.0,53.8,32.9,45.0,180.0,15,0,45.4,16000.0,1019.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,39.12,-75.47,"{""x"": -8401281.9702, ""y"": 4738875.1483, ""spati..."
Florida,15,KMKY,2021-02-24 16:04:00,Marco Island,United States Of America,2.0,78.8,69.8,74.0,170.0,15,0,NaN,16000.0,1022.0,Scattered Clouds at 580 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,82.5,26.00,-81.67,"{""x"": -9091462.8131, ""y"": 2999080.943500001, ""..."


In [236]:
sdfUSA_grouped.last().head(10)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE
State,,,,,,,,,,,,,,,,,,,,,,
Alabama,4995,KPRN,2021-02-24 14:58:00,Greenville,United States Of America,137.0,57.0,35.1,44.0,0.0,6,0,NaN,16000.0,1022.0,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,3-Hour Pressure Increasing: 1.4 mb,78.8,31.85,-86.62,"{""x"": -9642494.2925, ""y"": 3743636.8615999967, ..."
Alaska,5000,PANW,2021-02-24 14:56:00,New Stuyahok,United States Of America,100.0,26.1,26.1,100.0,100.0,13,0,17.5,6400.0,1002.0,"Few Clouds at 340 meters AGL, Scattered Clouds...","Light Snow, Mist",* Maintenance REQUIRED - readings may be compr...,NaN,59.45,-157.32,"{""x"": -17512782.2916, ""y"": 8278291.432099998, ..."
Arizona,3958,KRYN,2021-02-24 15:45:00,Tucson/Ryan Fld,United States Of America,731.0,51.8,12.2,20.0,100.0,13,0,40.2,16000.0,1019.3,Sky Clear below 3700 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,32.15,-111.17,"{""x"": -12375387.7915, ""y"": 3783016.603500001, ..."
Arkansas,4146,KMWT,2021-02-24 15:14:00,Mount Ida,United States Of America,220.0,57.0,50.0,77.0,0.0,7,0,NaN,16000.0,1014.2,"Broken Clouds at 400 meters AGL, Broken Clouds...",Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,34.55,-93.58,"{""x"": -10417277.9484, ""y"": 4102894.8566000015,..."
California,5043,KNFG,2021-02-24 14:55:00,Camp Pendleton,United States Of America,21.0,39.9,28.9,65.0,0.0,0,0,60.5,16000.0,1017.9,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,33.30,-117.35,"{""x"": -13063342.2446, ""y"": 3935191.9821000025,..."
Colorado,4994,KFCS,2021-02-24 14:59:00,Fort Carson,United States Of America,1789.0,28.4,17.6,63.0,170.0,26,0,16.5,16000.0,1020.3,Broken Clouds at 910 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,38.67,-104.77,"{""x"": -11662943.0504, ""y"": 4674511.5361, ""spat..."
Connecticut,3402,KBDL,2021-02-24 15:51:00,Windsor Locks,United States Of America,52.0,44.1,27.0,51.0,300.0,15,0,39.0,16000.0,1015.9,"Few Clouds at 1200 meters AGL, Few Clouds at 3...",No significant weather present at this time.,Wind Shift At 15:30Z,NaN,41.93,-72.68,"{""x"": -8090700.5909, ""y"": 5150499.544100001, ""..."
Delaware,3382,KILG,2021-02-24 15:51:00,Wilmington,United States Of America,22.0,48.9,33.1,54.0,140.0,13,0,45.4,16000.0,1018.9,Sky Clear below 3700 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,39.67,-75.60,"{""x"": -8415753.504, ""y"": 4818102.8226, ""spatia..."
Florida,5047,KNIP,2021-02-24 14:53:00,Jacksonville Nas,United States Of America,3.0,62.1,51.1,67.0,0.0,0,0,NaN,16000.0,1023.4,Scattered Clouds at 1000 meters AGL,No significant weather present at this time.,3-Hour Pressure Increasing: 1.5 mb,77.0,30.23,-81.67,"{""x"": -9091462.8131, ""y"": 3533148.591300003, ""..."


### Selection and filtering

[[back to top]](#Table-of-Contents)

Functions of descriptive statistic like `sum()`, `count()`, `max()`, `min()`, `mean()` can be quickly applied to the `GroupBy` object to obtain summary statistics for each group – an immensely useful function. The same statement is valid also for functions like `describe()` which return general information about an object. 

Let’s show a few examples of descriptive statistic function application to the `GroupBy` object

In [237]:
sdfUSA_grouped.count().head(10)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE
State,,,,,,,,,,,,,,,,,,,,,,
Alabama,41,41,41,41,41,41,41,41,41,41,41,41,0,40,41,40,41,5,1,41,41,41
Alaska,128,128,128,128,128,128,125,120,120,123,128,128,123,125,127,124,128,73,0,128,128,128
Arizona,42,42,42,42,42,42,42,42,42,42,42,42,14,41,42,42,42,5,0,42,42,42
Arkansas,36,36,36,36,36,36,36,36,36,35,36,36,0,35,36,36,36,3,0,36,36,36
California,131,131,131,131,131,131,130,130,130,131,131,131,78,130,131,130,131,18,0,131,131,131
Colorado,64,64,64,64,64,64,64,64,64,64,64,64,64,63,63,63,64,4,0,64,64,64
Connecticut,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,10,1,0,10,10,10
Delaware,4,4,4,4,4,4,4,4,4,4,4,4,1,4,4,4,4,3,0,4,4,4
Florida,95,95,95,95,95,93,94,94,94,94,95,95,0,94,95,94,95,22,52,95,95,95


In [238]:
sdfUSA_grouped.sum().add_prefix('sum_of_').head(10)
# calculate the sum only for numeric columns, 
# all columns with other types will be ignored and missed
# we have add also common prefics to all columns. It's very convenient

,sum_of_OBJECTID,sum_of_ELEVATION,sum_of_TEMP,sum_of_DEW_POINT,sum_of_R_HUMIDITY,sum_of_WIND_DIRECT,sum_of_WIND_SPEED,sum_of_WIND_GUST,sum_of_WIND_CHILL,sum_of_VISIBILITY,sum_of_PRESSURE,sum_of_HEAT_INDEX,sum_of_LATITUDE,sum_of_LONGITUDE
State,,,,,,,,,,,,,,
Alabama,90258,5525.0,2572.8,1513.7,1614.0,5780.0,367,0,0.0,634000.0,41874.9,78.8,1337.00,-3549.71
Alaska,231927,9685.0,1597.3,1254.4,10028.0,15270.0,2031,755,642.1,1323300.0,127976.0,0.0,7947.34,-19448.38
Arizona,100031,39608.0,2132.7,716.2,1178.0,4190.0,316,65,649.4,649000.0,42791.4,0.0,1417.80,-4695.26
Arkansas,88343,6070.0,2098.6,1573.4,2131.0,6140.0,547,199,0.0,556000.0,36545.2,0.0,1268.15,-3332.84
California,333401,39875.0,6184.2,4037.0,7379.0,14320.0,772,218,3907.9,2208100.0,133672.3,0.0,4729.19,-15665.82
Colorado,150536,132024.0,1656.1,754.6,3615.0,9180.0,839,314,1575.0,1008000.0,64253.1,0.0,2505.33,-6758.69
Connecticut,28734,632.0,455.0,292.6,533.0,1710.0,127,50,414.8,160000.0,10169.2,0.0,414.92,-727.51
Delaware,10097,60.0,210.3,132.0,190.0,700.0,56,0,45.4,64000.0,4077.8,0.0,156.69,-302.04
Florida,236043,1465.0,6726.3,5298.6,5646.0,9920.0,899,50,0.0,1477600.0,97148.2,4080.8,2703.97,-7832.29


In [239]:
sdfUSA.groupby(['State'])['TEMP'].mean()
# calculate sum for a single column
# it works more quickly then movies.groupby(['State']).mean()['TEMP'] becuase it sheads the data it does not need before the mean operation

State
Alabama               62.751220
Alaska                12.778400
Arizona               50.778571
Arkansas              58.294444
California            47.570769
Colorado              25.876562
Connecticut           45.500000
Delaware              52.575000
Florida               71.556383
Georgia               65.013924
Guam                  78.200000
Hawaii                68.523077
Idaho                 22.737500
Illinois              47.236207
Indiana               48.246341
Iowa                  35.655000
Johnston/Wake/Xmas    76.000000
Kansas                40.566667
Kentucky              60.521053
Louisiana             64.941176
Maine                 40.694118
Marshall Islands      79.300000
Maryland              51.409524
Massachusetts         45.747826
Michigan              39.309333
Minnesota             26.885000
Mississippi           62.759375
Missouri              48.972093
Montana               23.936842
Nebraska              34.484211
Nevada                35.568182
Ne

In [240]:
sdfUSA.groupby(['State', 'STATION_NAME']).sum()
# it is possible to group by many columns
# pay your attention that the changing of the order of 
# columns for grouping in 
# its list also changes the result table

OBJECTID  ELEVATION  ...  LATITUDE  LONGITUDE
State   STATION_NAME                           ...                     
Alabama Alabaster             3131      172.0  ...     33.18     -86.78
        Albertville Muni      2472      313.0  ...     34.23     -86.25
        Alexander/Russel      2443      198.0  ...     32.92     -85.97
        Andalusia/Opp Ar      1533       93.0  ...     31.32     -86.40
        Anniston              3247      187.0  ...     33.58     -85.85
...                            ...        ...  ...       ...        ...
Wyoming Torrington            2813     1285.0  ...     42.07    -104.15
        Warren Air Base       4997     1877.0  ...     41.13    -104.87
        Wheatland             2231     1452.0  ...     42.05    -104.93
        Worland               2787     1281.0  ...     43.97    -107.95
        Yellowstone Lake      1343     2377.0  ...     44.55    -110.42

[2364 rows x 14 columns]

It is possible also to show all groups

In [241]:
sdfUSA_grouped.groups

{'Alabama': [1132, 1147, 1150, 1154, 1161, 1180, 1345, 1372, 1381, 1450, 1470, 1532, 1535, 1607, 1721, 1728, 1818, 1929, 2015, 2019, 2222, 2297, 2314, 2427, 2442, 2471, 2514, 2515, 2563, 2814, 2822, 2944, 2966, 3037, 3059, 3130, 3151, 3220, 3223, 3246, 4994], 'Alaska': [1, 2, 7, 8, 9, 10, 17, 23, 36, 572, 1125, 1126, 1186, 1205, 1211, 1212, 1213, 1214, 1215, 1216, 1217, 1218, 1219, 1220, 1221, 1222, 1223, 1224, 1225, 1226, 1227, 1228, 1229, 1230, 1231, 1232, 1233, 1234, 1235, 1236, 1237, 1238, 1239, 1240, 1241, 1242, 1243, 1244, 1245, 1246, 1247, 1248, 1249, 1250, 1251, 1252, 1253, 1254, 1255, 1256, 1257, 1258, 1259, 1260, 1261, 1262, 1263, 1264, 1265, 1266, 1267, 1268, 1269, 1550, 1551, 1552, 1553, 1554, 1556, 1557, 1558, 1559, 2628, 2629, 2741, 2747, 2748, 2749, 2750, 2751, 2752, 2753, 2754, 2755, 2756, 2757, 2758, 2759, 2760, 2761, ...], 'Arizona': [1109, 1145, 1148, 1155, 1160, 1163, 1192, 1197, 1418, 1499, 1523, 1639, 1697, 1719, 1837, 1839, 2007, 2088, 2313, 2365, 2412, 2420, 246

and display the content only of necessary group

In [242]:
sdfUSA_grouped.get_group("Mississippi")

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE,State
1144,1145,KMEI,2021-02-24 15:58:00,Meridian/Key Fld,United States Of America,90.0,61.0,42.1,50.0,200.0,15,0,NaN,16000.0,1020.6,Sky Clear below 3700 meters AGL,No significant weather present at this time.,None,NaN,32.33,-88.75,"{""x"": -9879604.8079, ""y"": 3806706.633100003, ""...",Mississippi
1194,1195,KNJW,2021-02-24 15:57:00,Meridian Nas,United States Of America,165.0,62.1,30.0,30.0,210.0,22,0,NaN,16000.0,1018.6,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,32.78,-88.82,"{""x"": -9887397.1723, ""y"": 3866138.9184999987, ...",Mississippi
1340,1341,KPIB,2021-02-24 15:56:00,Pine Belt Rgnl,United States Of America,91.0,64.0,48.9,58.0,180.0,13,0,NaN,16000.0,1021.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,NaN,31.47,-89.33,"{""x"": -9944170.1126, ""y"": 3693939.2009000033, ...",Mississippi
1361,1362,KNMM,2021-02-24 15:56:00,Meridian Nas,United States Of America,94.0,63.0,39.9,43.0,220.0,13,0,NaN,16000.0,1020.3,Sky Clear below 3700 meters AGL,No significant weather present at this time.,None,NaN,32.53,-88.53,"{""x"": -9855114.5199, ""y"": 3833084.206799999, ""...",Mississippi
1432,1433,KHEZ,2021-02-24 15:56:00,Natchez/Hardy,United States Of America,82.0,66.0,51.1,59.0,200.0,20,0,NaN,16000.0,1019.3,Broken Clouds at 1200 meters AGL,Automated observation with no human augmentati...,None,NaN,31.62,-91.30,"{""x"": -10163469.5094, ""y"": 3713532.4341999963,...",Mississippi
1440,1441,KGTR,2021-02-24 15:56:00,Golden/Columbus,United States Of America,80.0,62.1,32.0,32.0,190.0,19,0,NaN,16000.0,1019.6,Sky Clear below 3700 meters AGL,No significant weather present at this time.,None,NaN,33.45,-88.58,"{""x"": -9860680.4945, ""y"": 3955187.3994000033, ...",Mississippi
1502,1503,KCBM,2021-02-24 15:56:00,Columbus Afb,United States Of America,59.0,61.5,29.8,30.0,190.0,11,0,NaN,16000.0,1018.9,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,33.65,-88.45,"{""x"": -9846208.9607, ""y"": 3981901.842100002, ""...",Mississippi
1513,1514,KBIX,2021-02-24 15:56:00,Biloxi/Keesler,United States Of America,4.0,66.2,52.0,60.0,140.0,15,0,NaN,16000.0,1021.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,30.42,-88.92,"{""x"": -9898529.1213, ""y"": 3557651.9789000005, ...",Mississippi
1611,1612,KVKS,2021-02-24 15:55:00,Vicksburg,United States Of America,31.0,64.4,48.2,56.0,210.0,11,26,NaN,16000.0,1018.9,Scattered Clouds at 1300 meters AGL,Automated observation with no human augmentati...,None,NaN,32.23,-90.93,"{""x"": -10122281.2978, ""y"": 3793539.7114000022,...",Mississippi
1622,1623,KUTA,2021-02-24 15:55:00,Tunica,United States Of America,58.0,57.2,37.4,47.0,190.0,22,0,NaN,16000.0,1016.9,Scattered Clouds at 940 meters AGL,Automated observation with no human augmentati...,None,NaN,34.68,-90.35,"{""x"": -10057715.9932, ""y"": 4120479.0078999996,...",Mississippi


In [243]:
sdfUSA_grouped.get_group("Mississippi").mean()

D66140B0-0277-45FF-AE18-A787BE3D4AAF:1: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.


OBJECTID        2241.030303
ELEVATION         73.212121
TEMP              62.759375
DEW_POINT         42.293750
R_HUMIDITY        48.968750
WIND_DIRECT      182.500000
WIND_SPEED        15.060606
WIND_GUST          2.909091
WIND_CHILL              NaN
VISIBILITY     16000.000000
PRESSURE        1019.324242
HEAT_INDEX              NaN
LATITUDE          32.644242
LONGITUDE        -89.681515
dtype: float64

We have just one helpful and powerful option for you! It is `filter()`.

Suppose you need remain only those groups where there are less than 5 items. Sure, you may hardly calculate amount of items in each group and then remove not satisfying for condition. But it can be done very easily.

In [244]:
sdfUSA_grouped.filter(lambda x: len(x) < 5).sort_values("State")

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE,State
1484,1485,KDOV,2021-02-24 15:56:00,Dover Afb,United States Of America,6.0,53.8,32.9,45.0,180.0,15,0,NaN,16000.0,1019.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,39.12,-75.47,"{""x"": -8401281.9702, ""y"": 4738875.1483, ""spati...",Delaware
2534,2535,K33N,2021-02-24 15:55:00,Cheswold/Dover,United States Of America,17.0,53.6,32.0,44.0,190.0,11,0,NaN,16000.0,1019.6,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,NaN,39.22,-75.60,"{""x"": -8415753.504, ""y"": 4753233.857000001, ""s...",Delaware
2694,2695,KGED,2021-02-24 15:54:00,Georgetown,United States Of America,15.0,54.0,34.0,47.0,190.0,17,0,NaN,16000.0,1020.0,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,38.68,-75.37,"{""x"": -8390150.0211, ""y"": 4675937.423600003, ""...",Delaware
3381,3382,KILG,2021-02-24 15:51:00,Wilmington,United States Of America,22.0,48.9,33.1,54.0,140.0,13,0,45.4,16000.0,1018.9,Sky Clear below 3700 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,39.67,-75.60,"{""x"": -8415753.504, ""y"": 4818102.8226, ""spatia...",Delaware
1210,1211,PGUA,2021-02-24 15:56:00,Mariana/Andersen,United States Of America,159.0,77.4,71.1,81.0,90.0,20,0,NaN,16000.0,1009.8,Scattered Clouds at 520 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,13.57,144.92,"{""x"": 16132420.605800003, ""y"": 1524929.4087000...",Guam
2626,2627,PGUM,2021-02-24 15:54:00,Agana,United States Of America,74.0,79.0,71.1,77.0,70.0,15,0,NaN,16000.0,1009.5,"Few Clouds at 610 meters AGL, Few Clouds at 10...",No significant weather present at this time.,Hourly Precipitation Amount: trace,83.2,13.48,144.80,"{""x"": 16119062.266900003, ""y"": 1514624.8880999...",Guam
570,571,PLCH,2021-02-24 16:00:00,Christmas/Cassid,United States Of America,5.0,75.2,73.4,94.0,90.0,9,0,NaN,10000.0,1006.9,Few Clouds at 610 meters AGL,No significant weather present at this time.,None,NaN,1.98,-157.35,"{""x"": -17516121.8763, ""y"": 220456.47509999946,...",Johnston/Wake/Xmas
1123,1124,PWAK,2021-02-24 15:58:00,Wake Island Airf,United States Of America,7.0,76.8,68.4,75.0,110.0,7,0,NaN,10000.0,1010.8,Few Clouds at 790 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,80.1,19.28,166.62,"{""x"": 18548053.556, ""y"": 2187929.2760000005, ""...",Johnston/Wake/Xmas
571,572,PKWA,2021-02-24 16:00:00,Kwajalein/Buchol,United States Of America,6.0,79.3,75.2,87.0,80.0,28,0,NaN,16000.0,1006.8,"Scattered Clouds at 1500 meters AGL, Scattered...",Automated observation with no human augmentati...,None,NaN,8.72,167.72,"{""x"": 18670504.995800003, ""y"": 974475.15549999...",Marshall Islands
1200,1201,TJSJ,2021-02-24 15:56:00,San Juan/Wfo,United States Of America,3.0,84.0,64.9,53.0,80.0,28,0,NaN,16000.0,1020.0,Few Clouds at 1200 meters AGL,No significant weather present at this time.,None,85.8,18.43,-66.02,"{""x"": -7349312.782199999, ""y"": 2087941.2272000...",Puerto Rico


One more example:

In [245]:
sdfUSA_grouped.filter(lambda x: x['TEMP'].mean() > 50 and x['R_HUMIDITY'].mean() < 50)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,SHAPE,State
962,963,KW99,2021-02-24 16:00:00,Petersburg,United States Of America,288.0,54.3,33.1,44.0,0.0,0,0,NaN,16000.0,1015.9,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,NaN,39.00,-79.15,"{""x"": -8810937.6963, ""y"": 4721671.5726, ""spati...",West Virginia
970,971,KMOR,2021-02-24 16:00:00,Morristown/Moore,United States Of America,393.0,59.0,33.8,39.0,250.0,11,0,NaN,14000.0,1018.6,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,NaN,36.18,-83.37,"{""x"": -9280705.9474, ""y"": 4325417.444600001, ""...",Tennessee
1109,1110,KNOZ,2021-02-24 15:59:00,Wellton,United States Of America,78.0,60.1,27.0,28.0,150.0,15,0,NaN,NaN,1016.9,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,32.53,-114.50,"{""x"": -12746081.695799999, ""y"": 3833084.206799...",Arizona
1110,1111,KNGU,2021-02-24 15:59:00,Norfolk Nas/Cham,United States Of America,3.0,57.9,32.0,37.0,170.0,24,37,NaN,16000.0,1021.0,"Scattered Clouds at 910 meters AGL, Broken Clo...",No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,36.93,-76.27,"{""x"": -8490337.5628, ""y"": 4429354.176200002, ""...",Virginia
1132,1133,KSXS,2021-02-24 15:58:00,Ft Rucker/Shell,United States Of America,120.0,64.6,43.5,46.0,240.0,9,0,NaN,14000.0,1022.0,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,None,NaN,31.37,-85.85,"{""x"": -9556778.2846, ""y"": 3680894.4886000007, ...",Alabama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4994,4995,KPRN,2021-02-24 14:58:00,Greenville,United States Of America,137.0,57.0,35.1,44.0,0.0,6,0,NaN,16000.0,1022.0,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,3-Hour Pressure Increasing: 1.4 mb,NaN,31.85,-86.62,"{""x"": -9642494.2925, ""y"": 3743636.8615999967, ...",Alabama
5000,5001,KVAD,2021-02-24 14:56:00,Moody Afb/Valdos,United States Of America,70.0,63.7,38.3,39.0,0.0,0,0,NaN,16000.0,1023.0,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,NaN,30.97,-83.20,"{""x"": -9261781.634, ""y"": 3628853.6849000007, ""...",Georgia
5004,5005,KLFI,2021-02-24 14:56:00,Langley Afb/Hamp,United States Of America,2.0,52.9,35.2,51.0,120.0,19,0,NaN,16000.0,1020.6,Scattered Clouds at 5500 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,NaN,37.08,-76.33,"{""x"": -8497016.7323, ""y"": 4450263.629600003, ""...",Virginia
5006,5007,KFCI,2021-02-24 14:56:00,Richmond/Chester,United States Of America,63.0,51.1,30.0,44.0,150.0,9,0,NaN,16000.0,1020.0,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,3-Hour Pressure Decreasing: -0.4 mb,NaN,37.40,-77.52,"{""x"": -8629486.9263, ""y"": 4495009.258599997, ""...",Virginia


### Aggregation

[[back to top]](#Table-of-Contents)

We have seen previously, pandas allows to apply any function to a Series or to a DataFrame. Grouping is not an exception and you may use the same method `apply()` directly with `GroupBy` objects.  Aggregation can be done via the `aggregate()` or equivalently `agg()` method

In [246]:
sdfUSA_grouped.agg(np.sum).head(10)
# or state_grouped.aggregate(np.sum)

,OBJECTID,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,HEAT_INDEX,LATITUDE,LONGITUDE
State,,,,,,,,,,,,,,
Alabama,90258,5525.0,2572.8,1513.7,1614.0,5780.0,367,0,0.0,634000.0,41874.9,78.8,1337.00,-3549.71
Alaska,231927,9685.0,1597.3,1254.4,10028.0,15270.0,2031,755,642.1,1323300.0,127976.0,0.0,7947.34,-19448.38
Arizona,100031,39608.0,2132.7,716.2,1178.0,4190.0,316,65,649.4,649000.0,42791.4,0.0,1417.80,-4695.26
Arkansas,88343,6070.0,2098.6,1573.4,2131.0,6140.0,547,199,0.0,556000.0,36545.2,0.0,1268.15,-3332.84
California,333401,39875.0,6184.2,4037.0,7379.0,14320.0,772,218,3907.9,2208100.0,133672.3,0.0,4729.19,-15665.82
Colorado,150536,132024.0,1656.1,754.6,3615.0,9180.0,839,314,1575.0,1008000.0,64253.1,0.0,2505.33,-6758.69
Connecticut,28734,632.0,455.0,292.6,533.0,1710.0,127,50,414.8,160000.0,10169.2,0.0,414.92,-727.51
Delaware,10097,60.0,210.3,132.0,190.0,700.0,56,0,45.4,64000.0,4077.8,0.0,156.69,-302.04
Florida,236043,1465.0,6726.3,5298.6,5646.0,9920.0,899,50,0.0,1477600.0,97148.2,4080.8,2703.97,-7832.29


Sure, above variant is equivalent to `state_grouped.sum()`.
One of the advantages of `agg()` method is the possibility of application of many functions to one and same dataset at once

In [255]:
# at first let's write the function which returns amount of notnull values 
def count_not_null(x):
    return len([i for i in x if pd.notnull(i)])
sdfUSA_grouped[['TEMP', 'R_HUMIDITY', "WIND_SPEED"]].agg([np.sum, np.mean, count_not_null]).head(10)

TEMP             ... WIND_SPEED               
                sum       mean  ...       mean count_not_null
State                           ...                          
Alabama      2572.8  62.751220  ...   8.951220             41
Alaska       1597.3  12.778400  ...  15.867188            128
Arizona      2132.7  50.778571  ...   7.523810             42
Arkansas     2098.6  58.294444  ...  15.194444             36
California   6184.2  47.570769  ...   5.893130            131
Colorado     1656.1  25.876562  ...  13.109375             64
Connecticut   455.0  45.500000  ...  12.700000             10
Delaware      210.3  52.575000  ...  14.000000              4
Florida      6726.3  71.556383  ...   9.463158             95
Georgia      5136.1  65.013924  ...   5.936709             79

[10 rows x 9 columns]

Sure, you may apply many functions to whole DataFrame at once

Another simple aggregation example is to compute the size of each group. This is included in `GroupBy` as the `size()` method. It returns a Series whose indexes are the group names and whose values are the sizes of each group.

In [257]:
sdfUSA_grouped.size()

State
Alabama                41
Alaska                128
Arizona                42
Arkansas               36
California            131
Colorado               64
Connecticut            10
Delaware                4
Florida                95
Georgia                79
Guam                    2
Hawaii                 13
Idaho                  24
Illinois               58
Indiana                41
Iowa                   60
Johnston/Wake/Xmas      2
Kansas                 55
Kentucky               38
Louisiana              51
Maine                  17
Marshall Islands        1
Maryland               22
Massachusetts          23
Michigan               75
Minnesota             100
Mississippi            33
Missouri               43
Montana                38
Nebraska               39
Nevada                 22
New Hampshire          13
New Jersey             17
New Mexico             32
New York               46
North Carolina         69
North Dakota           36
Ohio                   56
Oklaho

>### Exercise 1

> Read in the Gulf_and_Caribbean_Marine_Ecoregions.shp file (loaded into map 1) and also included in the data package.
> Use the groupby functions you learned about to create a dataframe that contains the total area (Shape_Area column) for each of unique values in the Lat_Zone field.


In [258]:
# type your code here

